# Autokodery
### Jakub Janaszkiewicz – Warsztaty KNDS MiNI – 07.03.2019

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
%matplotlib inline
mpl.rcParams['grid.color'] = 'None'

Za dane posłuży nam zbiór Olivetti faces - 400 zdjęć twarzy o różnej mimice (po 10 na każdą z 40 osób). Żeby powiększyć trochę rozmiar zbioru posłużymy się prostą techniką *data augmentation* i dodamy odbicia lustrzane zdjęć. W sumie da nam to 800 punktów danych.

In [ ]:
# Załadowanie zbioru danych
olivetti = fetch_olivetti_faces(download_if_missing=True, shuffle=True, random_state=1337)
X, y = olivetti.images, olivetti.target

# Data augmentation - odbicie lustrzane
X = np.concatenate([X, X[::-1, :]]).reshape((2 * X.shape[0], 64, 64))
y = np.concatenate([y, y])

print("Wymiary zbioru danych:", X.shape)
plt.imshow(X[np.random.randint(X.shape[0])], cmap='gray')
plt.title("Losowa twarz ze zbioru")
plt.show()

Do łatwego prototypowania sieci neuronowych użyjemy biblioteki **Keras**. Przy budowaniu płaskich modeli możemy posłuzyć się wartstwami *Flatten* i *Reshape* aby nie musieć za każdym razem ręcznie spłaszczać i kwadracić obrazków.

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Activation, Flatten, Reshape

## 1. Podstawowy autokoder
Autokodowanie (eng. *autoencoding*) to proces kompresji danych w którym funkcje kompresji i dekompresji są
1. Zależne od danych
2. Stratne
3. Uczące się automatycznie na danych (nie jest potrzebne ręczne wybieranie cech).

W każdym miejscu gdzie używane jest pojęcie *autokoder* obie funkcje implementowane są za pomocą sieci neuronowych.

### Stworzenie modelu
Najprostszym modelem autokodera jest jednowarstwowa sieć z warstwą ukrytą o mniejszym rozmiarze, niż rozmiar wejścia.

W tym przykładzie będziemy kodować obrazki o wymiarze 4096 (64 x 64) z użyciem warstwy ukrytej o rozmiarze 100 neuronów.

In [ ]:
hidden_size1 = 100
model1 = Sequential(layers=[
    InputLayer(input_shape=(64, 64)),
    Flatten(),
    Dense(hidden_size1, activation='sigmoid'),
    Dense(4096, activation='sigmoid'),
    Reshape(target_shape=(64, 64))
])
model1.compile(loss='mse', optimizer='adam')
#model.summary()

In [ ]:
history1 = model1.fit(x=X, y=X, epochs=400, batch_size=80, verbose=1, shuffle=True)

### Krzywa uczenia

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history1.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

### Wizualizacja ukrytej reprezentacji autokodera
Dla sieci neuronowych łatwo można zwizualizować na co zwracają uwagę neurony pierwszej warstwy ukrytej w danych wejściowych (tu: na które części zdjęcia). 

Korzystając z faktu, że dla danych wejściowych będących wektorami o długości n:

$$w_{(in,h)} \in M_{n}^{h}, \quad w_{(h, out)} \in M_{h}^{n}$$

możemy wziąć wagi należące do danego neuronu (jedną kolumnę macierzy wag) i przedstawić ich wartości w postaci obrazka.

#### Warstwa kodująca

In [ ]:
latent = model1.layers[1]
hidden = latent.get_weights()[0]
print("Wymiary macierzy wag:", hidden.shape)
offset = np.random.randint(0, hidden_size1)

fig = plt.figure(figsize=(20, 8))
rows, cols = 2, 4
for i in range(rows):
    for j in range(cols):
        k = cols * i + j
        ax = fig.add_subplot(rows, cols, k + 1)
        ax.imshow(hidden[:, k + offset].reshape(64, 64), cmap='gray')

fig.suptitle('Ukryta reprezentacja kodera', fontsize='x-large')
plt.show()

#### Warstwa dekodująca

In [ ]:
latent = model1.layers[2]
hidden = latent.get_weights()[0]
print("Wymiary macierzy wag:", hidden.shape)

fig = plt.figure(figsize=(20, 8))
rows, cols = 2, 4
for i in range(rows):
    for j in range(cols):
        k = cols * i + j
        ax = fig.add_subplot(rows, cols, k + 1)
        ax.imshow(hidden[k + offset, :].reshape(64, 64), cmap='gray')

fig.suptitle('Ukryta reprezentacja dekodera', fontsize='x-large')
plt.show()

#### Ekstrakcja dekodera i kodera
Biblioteka **Keras** pozwala na łatwe rozdzielenie naszego modelu na część kodującą i dekodującą. Tworzymy w tym celu nowe modele, podając w konstruktorach listę warstw z już przetrenowanego modelu.

In [ ]:
encoder1 = Sequential(model1.layers[:2])
encoder1.build(input_shape=(None, 64, 64))

decoder1 = Sequential(model1.layers[2:])
decoder1.build(input_shape=(None, hidden_size1))

#### Przekształcenia autokoderem
Zobaczmy jak nasz model poradził sobie z zadaniem kompresji.

In [ ]:
examples = 5
fig = plt.figure(figsize=(18, 12))
offset = np.random.randint(0, X.shape[0])
# oryginały
for k in range(examples):
    ax = fig.add_subplot(3, examples, k + 1)
    ax.imshow(X[k + offset], cmap='gray')
# kodowanie
for k in range(examples):
    ax = fig.add_subplot(3, examples, examples + k + 1)
    image = encoder1.predict(np.expand_dims(X[k + offset], 0))
    ax.imshow(image.reshape(10, 10), cmap='gray')
# przekształcenia
for k in range(examples):
    ax = fig.add_subplot(3, examples, 2 * examples + k + 1)
    image = model1.predict(np.expand_dims(X[k + offset], 0))[0]
    ax.imshow(image, cmap='gray')

plt.show()

## 2. Głęboki autokoder
Jak widać, o ile najprostsza architektura autokodera może sobie poradzić np. z MNISTem (*left as an exercise to the reader*), o tyle do kompresji zdjęć twarzy nie jest wystarczająca. Używając większej ilości warstw ukrytych model może nauczyć się bardziej złożonych cech obrazów.

### Stworzenie modelu
Użyjemy dwóch dodatkowych warstw, po jednej w części kodującej i dekodującej aby zachować symetrię. Tym razem celem będzie utajona reprezentacja o wymiarze 20.

In [ ]:
hidden_size2 = 20
model2 = Sequential(layers=[
    InputLayer(input_shape=(64, 64)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(hidden_size2, activation='relu'),
    Dense(256, activation='relu'),
    Dense(4096, activation='sigmoid'),
    Reshape(target_shape=(64, 64))
])
model2.compile(loss='mse', optimizer='adam')

In [ ]:
history2 = model2.fit(x=X, y=X, epochs=400, batch_size=80, verbose=1, shuffle=True)

### Krzywa uczenia

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history2.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

### Wizualizacja warstw ukrytych

#### Pierwsza warstwa kodująca

In [ ]:
latent = model2.layers[1]
hidden = latent.get_weights()[0]
offset = np.random.randint(0, hidden_size2)

fig = plt.figure(figsize=(20, 8))
rows, cols = 2, 4
for i in range(rows):
    for j in range(cols):
        k = cols * i + j
        ax = fig.add_subplot(rows, cols, k + 1)
        ax.imshow(hidden[:, k + offset].reshape(64, 64), cmap='gray')

fig.suptitle('Ukryta reprezentacja głębokiego kodera', fontsize='x-large')
plt.show()

#### Druga warstwa dekodująca

In [ ]:
latent = model2.layers[4]

hidden = latent.get_weights()[0]
fig = plt.figure(figsize=(20, 8))
rows, cols = 2, 4
for i in range(rows):
    for j in range(cols):
        k = cols * i + j
        ax = fig.add_subplot(rows, cols, k + 1)
        ax.imshow(hidden[k + offset, :].reshape(64, 64), cmap='gray')

fig.suptitle('Ukryta reprezentacja głębokiego dekodera', fontsize='x-large')
plt.show()

#### Ekstrakcja kodera i dekodera

In [ ]:
encoder2 = Sequential(model2.layers[:3])
encoder2.build(input_shape=(None, 64, 64))

decoder2 = Sequential(model2.layers[3:])
decoder2.build(input_shape=(None, hidden_size2))

#### Przekształcenia głębokim autokoderem

In [ ]:
examples = 5
fig = plt.figure(figsize=(18, 12))
offset = np.random.randint(0, X.shape[0])
# oryginały
for k in range(examples):
    ax = fig.add_subplot(3, examples, k + 1)
    ax.imshow(X[k + offset], cmap='gray')
# kodowanie
for k in range(examples):
    ax = fig.add_subplot(3, examples, examples + k + 1)
    image = encoder2.predict(np.expand_dims(X[k + offset], 0))
    ax.imshow(image.reshape(4, 5), cmap='gray')
# przekształcenia
for k in range(examples):
    ax = fig.add_subplot(3, examples, 2 * examples + k + 1)
    ax.imshow(model2.predict(np.expand_dims(X[k + offset], 0))[0], cmap='gray')

plt.show()

#### Generowanie nowych twarzy: podejście 1.
Techniki redukcji wymiarowości pozwalające na transformację odwrotną (np. *PCA*, autokodery) mogą być wykorzystywane do generowania nowych, nie widzianych wcześniej danych. Tak jak możemy próbkować $n$-wymiarową przestrzeń powstałą po zastosowaniu *PCA* i transformować wybrane wektory w górę, możemy zrobić podobnie w przypadku autokodera.

Zobaczmy do jakich przedziałów należą wartości ukrytych reprezentacji obrazów ze zbioru:

In [ ]:
encodings = encoder2.predict(X)
maxes = np.max(encodings, axis=0)

plt.bar(x=range(hidden_size2), height=maxes)
plt.xlabel('Neurony warstwy utajonej')
plt.xticks(range(hidden_size2))
plt.ylabel('RELu')
plt.title('Maksymalne wartości aktywacji')
plt.show()

Nie wszystkie neurony muszą być w ogóle aktywowane. W takim przypadku możemy spróbować jeszcze obniżyć wielkość warstwy ukrytej.

Do generowania nowych obserwacji można użyć np. biblioteki *ipywidgets* i suwaków, ale ponieważ Google Collaboratory ich nie obsługuje, wylosujemy po prostu wartości neuronów z przedziału \[0, max\] dla każdego i wrzucimy je w dekoder.

In [ ]:
#naive approach
fig = plt.figure(figsize=(20, 8))
rows, cols = 2, 4
for i in range(rows):
    for j in range(cols):
        k = cols * i + j
        ax = fig.add_subplot(rows, cols, k + 1)
        ax.imshow(decoder2.predict(np.random.beta(0.5, 0.5, size=(1, hidden_size2)) * maxes * 0.75)[0], cmap='gray')

fig.suptitle('Paskudne losowo wygenerowane twarze', fontsize='x-large')
plt.show()

## Zastosowania - redukcja wymiarowości
$n$ zdjęć $k$-wymiarowych możemy rozpatrywać jako $n$ punktów w $k$-wymiarowej przestrzeni

Weźmy dwie najpopularniejsze techniki redukcji wymiarowości:
* PCA (Principle Component Analysis) obracającą układ współrzędnych tak, aby maksymalizować w pierwszej kolejności wariancję pierwszej współrzędnej, następnie wariancję drugiej itd.
* T-SNE (T-distributed Stochastic Neighbor Embedding) - starającą się zminimalizować rozbieżność między dwoma rozkładami: podobieństwa punktów danych w przestrzeni wejściowej i podobieństwa punktów po redukcji

Autokodery składające się z warstw pełnych i o niewielkiej głębokości często uczą się reprezentacji wewnętrznej danych podobnej do głównych składowych wychwytywanych przez PCA. Kodery o bardziej rozbudowanej strukturze mają szansę nauczyć się ciekawszych rozkładów. 

Jednym z zastosowań autokoderów jest właśnie wizualizacja danych o wysokiej wymiarowości, zazwyczaj w połączeniu z PCA lub T-SNE.

#### Porównanie metod redukcji wymiarowości

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def compare_pca_tsne(encoder, the_shape):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X.reshape(800, 4096), y)
    dims = pca.n_components_

    fig = plt.figure(figsize=(20, 6))
    ax = fig.add_subplot(1, 4, 1)
    ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', s=10)
    ax.set_xlabel('Pierwsza składowa')
    ax.set_ylabel('Druga składowa')
    ax.set_title('PCA(X)')
    
    encodings = encoder.predict(X.reshape(the_shape))
    encodings_pca = pca.fit_transform(encodings)

    ax = fig.add_subplot(1, 4, 2)
    ax.scatter(encodings_pca[:, 0], encodings_pca[:, 1], c=y, cmap='viridis', s=10)
    ax.set_xlabel('Pierwsza składowa')
    ax.set_ylabel('Druga składowa')
    ax.set_title('PCA(AE(X))')

    tsne_enc = TSNE(n_components=2).fit_transform(X.reshape((800, 4096)))

    ax = fig.add_subplot(1, 4, 3)
    ax.scatter(tsne_enc[:, 0], tsne_enc[:, 1], c=y, cmap='viridis', s=10)
    ax.set_xlabel('Pierwszy wymiar')
    ax.set_ylabel('Drugi wymiar')
    ax.set_title('TSNE(X)')

    tsne_enc = TSNE(n_components=2).fit_transform(encodings)

    ax = fig.add_subplot(1, 4, 4)
    ax.scatter(tsne_enc[:, 0], tsne_enc[:, 1], c=y, cmap='viridis', s=10)
    ax.set_xlabel('Pierwszy wymiar')
    ax.set_ylabel('Drugi wymiar')
    ax.set_title('TSNE(AE(X))')
    
    pca.
    
    plt.show()
 

compare_pca_tsne(encoder2, (800, 64, 64))

#### Generowanie nowych twarzy: podejście 2

In [ ]:
def sample_PCA(pca, decoder, hidden_size, **kwargs):
    vector = np.zeros(pca.n_components_)
    i = 0
    for value in kwargs.values():
        vector[i] = value
        i += 1
    
    image = decoder.predict(pca.inverse_transform(vector).reshape(1, hidden_size)).reshape((64, 64))
    plt.imshow(image, cmap='gray')
        
def generate_PCA(encoder, decoder, hidden_size):  
    dims = 16
    pca = PCA(n_components = dims).fit(encoder.predict(X))
    fig = plt.figure(figsize=(20, 8))
    rows, cols = 2, 4
    for i in range(rows):
        for j in range(cols):
            k = cols * i + j
            ax = fig.add_subplot(rows, cols, k + 1)
            kwargs = dict(zip(['feature_' + str(t) for t in range(dims)], 2 * np.random.normal(size=dims)))
            sample_PCA(pca, decoder, hidden_size, **kwargs)
    fig.suptitle('Twarze wygenerowane z PCA', fontsize='x-large')
    plt.show()

X = X.reshape(800, 64, 64)
generate_PCA(encoder2, decoder2, hidden_size2)

## 3. Konwolucyjny autokoder
Oczywiście nikt nie broni zbudować autokodera z warstw konwolucyjnych. Model taki będzie się lepiej sprawował w przypadku danych, w których przestrzenne ustawienie jest istotne (np. obrazy właśnie, czy dźwięk).

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose

#### Stworzenie modelu

In [ ]:
hidden_size3 = 32
def createConvModel():
    model3 = Sequential()

    model3.add(InputLayer(input_shape=(64, 64, 1)))
    print(model3.output_shape)

    model3.add(Conv2D(filters=32, kernel_size=3))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(MaxPooling2D(pool_size=(2,2)))
    print(model3.output_shape)

    model3.add(Conv2D(filters=64, kernel_size=3))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(MaxPooling2D(pool_size=(2,2)))
    print(model3.output_shape)

    model3.add(Conv2D(filters=128, kernel_size=3))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(MaxPooling2D(pool_size=(2,2)))
    print(model3.output_shape)

    model3.add(Flatten())
    print(model3.output_shape)

    model3.add(Dense(hidden_size3))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(Dense(4608))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(Reshape((6, 6, 128)))
    print(model3.output_shape)

    model3.add(Conv2DTranspose(filters=128, kernel_size=5, strides=2))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(Conv2DTranspose(filters=64, kernel_size=3, strides=2))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(Conv2DTranspose(filters=32, kernel_size=2, strides=2))
    model3.add(Activation("relu"))
    print(model3.output_shape)

    model3.add(Conv2DTranspose(filters=1, kernel_size=3, strides=1))
    model3.add(Activation("sigmoid"))
    print(model3.output_shape)

    #model3.summary()
    model3.compile(loss='mse', optimizer='adam')
    return model3

model3 = createConvModel()

In [ ]:
X = X.reshape((800, 64, 64, 1))
history3 = model3.fit(x=X, y=X, epochs=200, batch_size=80, verbose=1)

#### Krzywa uczenia

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history3.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

#### Ekstrakcja kodera i dekodera

In [ ]:
encoder3 = Sequential(model3.layers[:11])
encoder3.build(input_shape=(None, 64, 64, 1))

decoder3 = Sequential(model3.layers[11:])
decoder3.build(input_shape=(None, hidden_size3))

#### Przekształcenia autokoderem

In [ ]:
examples = 5
offset = np.random.randint(0, X.shape[0] - examples)
fig = plt.figure(figsize=(20, 12))
# oryginały
for k in range(examples):
    ax = fig.add_subplot(3, examples, k + 1)
    ax.imshow(X[k + offset].reshape(64, 64), cmap='gray')
# kodowanie
for k in range(examples):
    ax = fig.add_subplot(3, examples, examples + k + 1)
    image = encoder3.predict(np.expand_dims(X[k + offset], 0))
    ax.imshow(image.reshape(4, 8), cmap='gray')
# przekształcenia
for k in range(examples):
    ax = fig.add_subplot(3, examples, 2 * examples + k + 1)
    ax.imshow(model3.predict(X[k + offset].reshape((1, 64, 64, 1))).reshape(64, 64), cmap='gray')

fig.suptitle('Przekształcenia autokoderem', fontsize='x-large')
plt.show()

#### Porównanie metod redukcji wymiarowości

In [ ]:
compare_pca_tsne(encoder3, (800, 64, 64, 1))

#### Generowanie nowych twarzy: podejście 3.

In [ ]:
X = X.reshape((800, 64, 64, 1))
generate_PCA(encoder3, decoder3, hidden_size3)

Na zbiorze danych zawierającym 800 zdjęć twarzy z frontu trudno uzyskać przekonujące wyniki. Tutaj materiał, który prezentuje tę samą technikę (szum -> PCA -> warstwa ukryta -> dekodowanie), [ale na znacznie większym zbiorze danych](https://youtu.be/4VAkrUNLKSo?t=351)

## Zastosowanie: odszumianie
Możemy zastosować autokoder przetrenowany na zaszumionych danych wejściowych aby otrzymać model potrafiący odszumiać dane podobne do tych ze zbioru treningowego.

In [ ]:
noise_factor = 0.05
X_noisy = X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X.shape)

plt.imshow(X_noisy[np.random.randint(X_noisy.shape[0])].reshape((64, 64)), cmap='gray')
plt.title("Losowa zaszumiona twarz ze zbioru")
plt.show()

#### Stworzenie modelu

In [ ]:
model4 = createConvModel()
X_noisy = X_noisy.reshape((800, 64, 64, 1))
X = X.reshape((800, 64, 64, 1))

In [ ]:
history4 = model4.fit(x=X_noisy, y=X, epochs=100, verbose=1, shuffle=True)

#### Krzywa uczenia

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history4.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

#### Ekstrakcja kodera i dekodera

In [ ]:
encoder4 = Sequential(model4.layers[:11])
encoder4.build(input_shape=(None, 64, 64, 1))

decoder4 = Sequential(model4.layers[11:])
decoder4.build(input_shape=(None, hidden_size3))

#### Odszumianie

In [ ]:
examples = 5
offset = np.random.randint(0, X.shape[0] - examples)
fig = plt.figure(figsize=(20, 12))
# zaszumione
for k in range(examples):
    ax = fig.add_subplot(4, examples, k + 1)
    ax.imshow(X_noisy[k + offset].reshape(64, 64), cmap='gray')
# kodowanie
for k in range(examples):
    ax = fig.add_subplot(4, examples, examples + k + 1)
    image = encoder4.predict(np.expand_dims(X_noisy[k + offset], 0))
    ax.imshow(image.reshape(4, 8), cmap='gray')
# przekształcenia
for k in range(examples):
    ax = fig.add_subplot(4, examples, 2 * examples + k + 1)
    ax.imshow(model4.predict(X_noisy[k + offset].reshape((1, 64, 64, 1))).reshape(64, 64), cmap='gray')
# oryginały
for k in range(examples):
    ax = fig.add_subplot(4, examples, 3 * examples + k + 1)
    ax.imshow(X[k + offset].reshape(64, 64), cmap='gray')

plt.show()

Odszumianie nie musi oznaczać tylko eliminacji białego szumu.
Inne przykłady:
* rozpoznawanie kart z [Magic the Gathering](https://hackernoon.com/a-deep-convolutional-denoising-autoencoder-for-image-classification-26c777d3b88e)
* [oczyszczanie obrazów](https://arxiv.org/pdf/1606.08921.pdf)
* wypełnianie dziur w obrazach ([image impainting](http://people.eecs.berkeley.edu/~pathak/context_encoder/))

## Wariacyjny autokoder
Widzieliśmy jak możemy próbować generować nowe obserwacje próbkując warstwę ukrytą różnych autokoderów. W praktyce jednak takie podejście nie jest stosowane. Modelami generatywnymi w kontekście autokoderów są **autokodery wariacyjne**. Budowa przypomina Przedstawione wcześniej głębokie autokodery, jednak zamiast mapować wejścia na neurony symbolizujące arbitralne liczby, mapujemy na neurony oznaczające parametry pewnego rozkładu prawdopodobieństwa, który potem możemy próbkować.

In [ ]:
from keras import backend as K
from keras.layers import Input, Lambda
from keras.models import Model

#### Stworzenie modelu

Najpierw, utwórzmy część kodującą dane wejściowe do przestrzeni utajonej. Użyjemy w tym celu funkcyjnego API biblioteki **Keras**, pozwalającego na utworzenie niestandardowej konfiguracji warstw sieci (dwie warstwy na tym samym poziomie).

In [ ]:
hidden_size5 = 512
latent_size = 2

input_layer = Input(batch_shape=(None, 4096))
hidden_layer = Dense(hidden_size5, activation='relu')(input_layer)
z_mean = Dense(latent_size)(hidden_layer)
z_log_sigma = Dense(latent_size)(hidden_layer)

Używając tych parametrów możemy stworzyć generator podobnych danych z powstałej przestrzeni. Ponieważ nie możemy próbkować naszych rozkładów bezpośrednio (utracilibyśmy różniczkowalność) zastosujemy tzw. "reparametrizatoin trick":
![alt text](https://www.jeremyjordan.me/content/images/2018/03/Screen-Shot-2018-03-18-at-4.36.34-PM.png)

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# uwaga: "output_shape" nie jest konieczny przy używaniu backendu TensorFlow
# można też napisać `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling, output_shape=(latent_size,))([z_mean, z_log_sigma])

Na koniec przekształcamy próbkowane punkty z powrotem na odtworzone dane wejściowe

In [ ]:
decoder_h = Dense(hidden_size5, activation='relu')
decoder_mean = Dense(4096, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

Na koniec otrzymujemy 3 modele:

In [ ]:
# autokoder end-to-end
vae = Model(input_layer, x_decoded_mean)

# koder, z danych wejściowych do przestrzeni utajonej
v_encoder = Model(input_layer, z_mean)

# generator, z przestrzeni utajonej w odtworzone dane wejściowe
decoder_input = Input(shape=(latent_size,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

#### Trening

Trenujemy nasz model ze specjalną funkcją błędu: sumą błędu rekonstrukcji oraz dywergencji Kullbacka-Leiblera (KL divergence) (określa rozbieżność między dwoma rozkładami prawdopodobieństwa).

In [ ]:
from keras.objectives import mse

def vae_loss(x, x_decoded_mean):
    xent_loss = mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    kl_loss /= 800
    return xent_loss + kl_loss / 2

vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()

Trenujemy nasz model

In [ ]:
X = X.reshape((800, 4096))
history5 = vae.fit(x=X, y=X, epochs=200, shuffle=True)

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax = fig.add_subplot(121)
ax.plot(history5.history['loss'])

plt.title('Learning curve')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.show()

#### Wizualizacja wyników

In [ ]:
def plot_results(models, data, batch_size=80):

    encoder, decoder = models
    x_test, y_test = data

    # wykres klas w przestrzeni utajonej
    z_mean = encoder.predict(x_test, batch_size=batch_size)
    plt.figure(figsize=(20, 12))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test, cmap='viridis')
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

    # 15x15 2D macierz twarzy
    n = 15
    image_size = 64
    figure = np.zeros((image_size * n, image_size * n))
    # liniowo rozmieszczone koordynaty odpowiadające wykresowi 2D
    # klas twarzy w p. utajonej
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(image_size, image_size)
            figure[i * image_size: (i + 1) * image_size,
                   j * image_size: (j + 1) * image_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = image_size // 2
    end_range = n * image_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, image_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='gray')
    plt.show()
    
plot_results(models=(v_encoder, generator), data=(X, y), batch_size=80)

Więcej materiałów na temat *VAE*:
* [film na YT](https://youtu.be/9zKuYvjFFS8) opisujący zasadę działania
* [ten post](http://kvfrans.com/variational-autoencoders-explained/) (w prostych słowach)
* i [ten post](https://www.jeremyjordan.me/variational-autoencoders/) z którego pochodzi obrazek, a temat jest dokładniej wytłumaczony